### Import libraries

In [2]:
import pandas as pd
import numpy as np
import os
import glob
import cv2
os.environ['KERAS_BACKEND'] = 'tensorflow'
import keras
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from PIL import Image
from matplotlib.pyplot import imshow


### Exploratory Analysis

In [3]:
image_filepaths = glob.glob('dataset-original/*')
classes = [] # list of files for each class
category_mapping = {} # map category name to integer

for i in range(len(image_filepaths)):
    classes.append(glob.glob(image_filepaths[i] + '/*.jpg'))
    category_mapping[i] = image_filepaths[i].split('/')[1]

In [4]:
print(category_mapping)

{0: 'paper', 1: 'metal', 2: 'cardboard', 3: 'trash', 4: 'glass', 5: 'plastic'}


In [5]:
print("Number of classes:",len(classes),'\n')

for idx,ic in enumerate(image_filepaths):
    print(ic.split('/')[1],":",len(classes[idx]),"images")

NUM_IMAGES = sum(len(c) for c in classes)
print("\nTotal number of images",NUM_IMAGES)

Number of classes: 6 

paper : 594 images
metal : 410 images
cardboard : 403 images
trash : 137 images
glass : 501 images
plastic : 482 images

Total number of images 2527


### Load images

In [6]:
HEIGHT,WIDTH = 150,150

In [7]:
X = np.zeros((NUM_IMAGES,HEIGHT,WIDTH,3))
y = np.zeros((NUM_IMAGES,1))
print(X.shape,y.shape)

(2527, 150, 150, 3) (2527, 1)


## IMPORTANT! ONLY LOADING 10 IMAGES FROM EACH CLASS NOW, PLS CHANGE THIS

In [8]:
k = 0
for idx,c in enumerate(classes): 
    for image in c[:10]: # TODO: CHANGE HERE
        im = Image.open(image).resize((HEIGHT,WIDTH))     
        X[k] = im
        y[k] = idx
        k+=1

### Basic CNN 

In [12]:
batch_size = 1
num_classes = 6
epochs = 2

# the data, split between train and test sets
X_train, X_test, y_train, y_test = train_test_split(X[:60], y[:60], test_size=0.20) ## TODO: CHANGE 60 here too

In [13]:
print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)

(48, 150, 150, 3) (12, 150, 150, 3) (48, 1) (12, 1)


In [14]:
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'training samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (48, 150, 150, 3)
48 training samples
12 test samples


In [15]:
print(y_train.shape,y_test.shape)

(48, 6) (12, 6)


In [16]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 48 samples, validate on 12 samples
Epoch 1/2
48/48 [==============================] - 51s 1s/step - loss: 12.5460 - acc: 0.2083 - val_loss: 13.4317 - val_acc: 0.1667
Epoch 2/2
48/48 [==============================] - 56s 1s/step - loss: 13.4317 - acc: 0.1667 - val_loss: 13.4317 - val_acc: 0.1667
Test loss: 13.431746482849121
Test accuracy: 0.1666666716337204
